## Table of contents
#### 1 noise removal: 
punctuation(dấu câu), stop words (stopwordsVN), URLs and Special Characters, date time, repeating chars
#### 2 normalization: 
Elongation, accent-marks((e.g., convert all variations of "á" to a single form)), từ ngữ không chính thống (slang, viết tắt) và các biến thể về chính tả, sửa lỗi chính tả, lowercase, num2word.
#### 3 word segmentation
#### 4 handle negation
#### 5 Xử lí label

In [2]:
pip install underthesea pyvi num2words py_vncorenlp pyvi

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 24.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 70.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.3/143.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 6.8 MB/s eta 0:00:0000:0100:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 47.1 MB/s eta 0:00:00
  Created wheel for py_vncorenlp: filename=py_vncorenlp-0.1.4-py3-none-any.whl size=4305 sha256=253234f6975d52cd2d4bc4f2749828a3555b2d47b4930faf50c661db07b50d9e
  Stored in directory: /root/.cache/pip/wheels/d5/d9/bf/62632cdb007c702a0664091e92a0bb1f18a2fcecbe962d9827
Successfully built py_vncorenlp
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np

import unicodedata
from underthesea import word_tokenize
import re
import string

## 1. Noise removal

In [4]:
# Xử lí dấu câu
def remove_punctuation(text):
    translator = str.maketrans(string.punctuation, ' ' * len(string.punctuation))
    text = text.translate(translator)
    return text

def remove_diacritics(text):
    text = unicodedata.normalize('NFD', text)
    text = ''.join(c for c in text if not unicodedata.combining(c))
    return text

In [5]:
# Stopwords
import requests

raw_url = 'https://raw.githubusercontent.com/stopwords/vietnamese-stopwords/master/vietnamese-stopwords.txt'
response = requests.get(raw_url)

if response.status_code == 200:
    content = response.text
else:
    print('Failed to retrieve the file from GitHub:', response.status_code)

# Stopwords
sw = content.split('\n')
def remove_stopword(text):
    text = " ".join(x for x in text.split() if x not in sw)
    return text

In [6]:
# # URLs and special character
# def clean_text(text):
#     '''Remove special character, remove links'''

#     text = re.sub('https?://\S+|www\.\S+', '', text)
    
#     text = re.sub('\[.*?\]', '', text)
# #     text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
#     text = text.replace(u'  ', u' ')
#     text = text.replace(u'"', u' ')
#     text = text.replace(u'️', u'')
    
#     return text

import re

def clean_text(text):
    # Remove URLs, special characters and date time

    # Remove URLs
    text = re.sub(r"https?://\S+", "", text)
    
    # Remove specific special characters
    text = re.sub(r"[!@#$%^&*(]", "", text)
    
    # Remove dates in the format dd/mm/yyyy or dd-mm-yyyy or dd.mm.yyyy
    text = re.sub(r'\b\d{1,2}[/-]\d{1,2}[/-]\d{2,4}\b', '', text)

    # Remove dates in the format dd_mm_yy
    text = re.sub(r'\b\d{1,2}[_]\d{1,2}[_]\d{2}\b', '', text)
    
    return text


In [7]:
#Xử lý Emoji
#Quy các icon về 2 loại emoj: Tích cực hoặc tiêu cực
def process_emojis(text):
    emojis_list = {
        "👹": "negative", "👻": "positive", "💃": "positive",'🤙': 'positive ', '👍': 'positive ',
        "💄": "positive", "💎": "positive", "💩": "positive","😕": "negative", "😱": "negative", "😸": "positive",
        "😾": "negative", "🚫": "negative",  "🤬": "negative","🧚": "positive", "🧡": "positive",'🐶':'positive ',
        '👎': 'negative ', '😣': 'negative ','✨': 'positive ', '❣': 'positive ','☀': 'positive ',
        '♥': 'positive ', '🤩': 'positive ', 'like': 'positive ', ':))': 'positive ', ':)': 'positive ',
        'he he': 'positive ','hehe': 'positive ','hihi': 'positive ', 'haha': 'positive ', 'hjhj': 'positive ',
        ' lol ': 'negative ',' cc ': 'negative ', 'huhu': 'negative ', '><': u'positive ',
        '💌': 'positive ', '🥰': 'positive ', '🙆' : 'positive ', '😅' : 'negative ',
        '🤒' : 'negative ', '🤨' : 'negative ', '🤦' : 'negative ', '😬' :'negative ',
        '🔋' : 'positive ', '💔' : 'negative ', '🤮' : 'negative ', '✋' : 'positive ',
        '🤣': 'positive ', '🖤': 'positive ', '🤤': 'positive ', ':(': 'negative ', '😢': 'negative ',
        '❤': 'positive ', '😍': 'positive ', '😘': 'positive ', '😪': 'negative ', '😊': 'positive ',
        '?': ' ? ', '😁': 'positive ', '💖': 'positive ', '😟': 'negative ', '😭': 'negative ',
        '💯': 'positive ', '💗': 'positive ', '♡': 'positive ', '💜': 'positive ', '🤗': 'positive ',
        '^^': 'positive ', '😨': 'negative ', '☺': 'positive ', '💋': 'positive ', '👌': 'positive ',
        '😖': 'negative ', '😀': 'positive ', ':((': 'negative ', '😡': 'negative ', '😠': 'negative ',
        '😒': 'negative ', '🙂': 'positive ', '😏': 'negative ', '😝': 'positive ', '😄': 'positive ',
        '😙': 'positive ', '😤': 'negative ', '😎': 'positive ', '😆': 'positive ', '💚': 'positive ',
        '✌': 'positive ', '💕': 'positive ', '😞': 'negative ', '😓': 'negative ', '️🆗️': 'positive ',
        '😉': 'positive ', '😂': 'positive ', ':v': 'positive ', '=))': 'positive ', '😋': 'positive ',
        '💓': 'positive ', '😐': 'negative ', ':3': 'positive ', '😫': 'negative ', '😥': 'negative ',
        '😃': 'positive ', '😬': ' 😬 ', '😌': ' 😌 ', '💛': 'positive ', '🤝': 'positive ', '🎈': 'positive ',
        '😗': 'positive ', '🤔': 'negative ', '😑': 'negative ', '🔥': 'negative ', '🙏': 'negative ',
        '🆗': 'positive ', '😻': 'positive ', '💙': 'positive ', '💟': 'positive ',
        '😚': 'positive ', '❌': 'negative ', '👏': 'positive ', ';)': 'positive ', '<3': 'positive ',
        '🌝': 'positive ',  '🌷': 'positive ', '🌸': 'positive ', '🌺': 'positive ',
        '🌼': 'positive ', '🍓': 'positive ', '🐅': 'positive ', '🐾': 'positive ', '👉': 'positive ',
        '💐': 'positive ', '💞': 'positive ', '💥': 'positive ', '💪': 'positive ',
        '💰': 'positive ',  '😇': 'positive ', '😛': 'positive ', '😜': 'positive ',
        '🙃': 'negative ', '🤑': 'positive ', '🤪': 'positive ','☹': 'negative ',  '💀': 'negative ',
        '😔': 'negative ', '😧': 'negative ', '😩': 'negative ', '😰': 'negative ', '😳': 'negative ',
        '😵': 'negative ', '😶': 'negative ', '🙁': 'negative ', '🎉': 'positive '}
    for emoji, label in emojis_list.items():
        text = text.replace(emoji, f'EMO{label.upper()}' )
        
    text = ' ' .join(text.split())
    return text

In [8]:
# Repeating chars
def cleaning_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)

In [9]:
def noise_removal(text):
    
    text = remove_punctuation(text)
    text = remove_diacritics(text)
    text = remove_stopword(text)
    text = clean_text(text)
    text = process_emojis(text)
    text = cleaning_repeating_char(text)
    
    return text

## 2. Normalization

In [10]:
# Remove các ký tự kéo dài: vd: đẹppppppp
def remove_elongated_chars(text):
    replacements = {
       'a' : 'àáảãạăằắẳẵặâầấẩẫậ' ,
       'e' : 'èéẻẽẹêềếểễệ' ,
       'i' : 'ìíỉĩị' ,
       'o' : 'òóỏõọôồốổỗộơờớởỡợ' ,
       'u' : 'ùúủũụưừứửữự' ,
       'y' : 'ỳýỷỹỵ' ,
       'd' : 'đ' ,
       'A' : 'ÀÁẢÃẠĂẰẮẲẴẶÂẦẤẨẪẬ' ,
       'E' : 'ÈÉẺẼẸÊỀẾỂỄỆ' ,
       'I' : 'ÌÍỈĨỊ' ,
       'O' : 'ÒÓỎÕỌÔỒỐỔỖỘƠỜỚỞỠỢ' ,
       'U' : 'ÙÚỦŨỤƯỪỨỬỮỰ' ,
       'Y' : 'ỲÝỶỸỴ' ,
       'D' : 'Đ' 
    }
    
    for char, replacements_str in replacements.items():
        pattern = rf"({char})\1+"
        text = re.sub(pattern, ' ' , text)
    pattern = rf"(\w)\1+"
    text = re.sub(pattern, r'\1', text)
    return text


In [11]:
#Sử dụng thư viện unicodedata để chuyển đổi các ký tự Unicode 
#tương đương thành dạng chuẩn. Ví dụ: "Hoà" thành "Hòa".
def normalize_unicode(text):
    return unicodedata.normalize("NFC", text)

In [12]:
#Chuẩn hóa từ vùng, từ viết tắt 
#Tạo từ điển ánh xạ các từ vựng cần chuẩn hóa sang dạng chuẩn. 
#Ví dụ: {"okie": "ok", "okey": "ok", "authentic": "chuẩn chính hãng"}.
#Sử dụng từ điển này để thay thế các từ vựng trong văn bản.

def normalize_sentiment_words(text):
    sentiment_word_map = {
        'ô kêi': ' ok ', 'okie': ' ok ', ' o kê ': ' ok ',
        'okey': ' ok ', 'ôkê': ' ok ', 'oki': ' ok ', ' oke ':  ' ok ',' okay':' ok ','okê':' ok ',
        ' tks ': u' cám ơn ', 'thks': u' cám ơn ', 'thanks': u' cám ơn ', 'ths': u' cám ơn ', 'thank': u' cám ơn ',
        '⭐': 'star ', '*': 'star ', '🌟': 'star ',
        'kg ': u' không ','not': u' không ', u' kg ': u' không ', '"k ': u' không ',' kh ':u' không ','kô':u' không ','hok':u' không ',' kp ': u' không phải ',u' kô ': u' không ', '"ko ': u' không ', u' ko ': u' không ', u' k ': u' không ', 'khong': u' không ', u' hok ': u' không ',
        'cute': u' dễ thương ', ' vs ': u' với ', 'wa': ' quá ', 'wá': u' quá', 'j': u' gì ', '“': ' ',
        ' sz ': u' cỡ ', 'size': u' cỡ ', u' đx ': u' được ', 'dk': u' được ', 'dc': u' được ', 'đk': u' được ',
        'đc': u' được ','authentic': u' chuẩn chính hãng ',u' aut ': u' chuẩn chính hãng ', u' auth ': u' chuẩn chính hãng ', 'store': u' cửa hàng ',
        'shop': u' cửa hàng ', 'sp': u' sản phẩm ', 'gud': u' tốt ','god': u' tốt ','wel done':' tốt ', 'good': u' tốt ', 'gút': u' tốt ',
        'sấu': u' xấu ','gut': u' tốt ', u' tot ': u' tốt ', u' nice ': u' tốt ', 'perfect': 'rất tốt', 'bt': u' bình thường ',
        'time': u' thời gian ', 'qá': u' quá ', u' ship ': u' giao hàng ', u' m ': u' mình ', u' mik ': u' mình ',
        'ể': 'ể', 'product': 'sản phẩm', 'quality': 'chất lượng','chat':' chất ', 'excelent': 'hoàn hảo', 'bad': 'tệ','fresh': ' tươi ','sad': ' tệ ',
        'date': u' hạn sử dụng ', 'hsd': u' hạn sử dụng ','quickly': u' nhanh ', 'quick': u' nhanh ','fast': u' nhanh ','delivery': u' giao hàng ',u' síp ': u' giao hàng ',
        'beautiful': u' đẹp tuyệt vời ', u' tl ': u' trả lời ', u' r ': u' rồi ', u' shopE ': u' cửa hàng ',u' order ': u' đặt hàng ',
        'chất lg': u' chất lượng ',u' sd ': u' sử dụng ',u' dt ': u' điện thoại ',u' nt ': u' nhắn tin ',u' tl ': u' trả lời ',u' sài ': u' xài ',u'bjo':u' bao giờ ',
        'thick': u' thích ', 'thik': u' thích ', u' sop ': u' cửa hàng ', u' shop ': u' cửa hàng ', 
        ' fb ': ' facebook ', ' face ': ' facebook ', ' very ': u' rất ',u'quả ng ':u' quảng  ',
        'dep': u' đẹp ',u' xau ': u' xấu ','delicious': u' ngon ', u'hàg': u' hàng ', u'qủa': u' quả ',
        'iu': u' yêu ','fake': u' giả mạo ', 'trl': 'trả lời',
        ' por ': u' tệ ',' poor ': u' tệ ', 'ib':u' nhắn tin ', 'rep':u' trả lời ',u'fback':' feedback ','fedback':' feedback '
    }
    for word, replacement in sentiment_word_map.items():
        text = text.replace(word, replacement)
    return text


In [13]:
#Chuẩn hóa số sao đánh giá/điểm số
def normalize_stars(text):
    stars_map = {
        '5star': 'positive ','5 sao': 'positive ','5sao': 'positive ', 'starstarstarstarstar': 'positive ',
        '4star': 'positive ','4 sao': 'positive ','4sao': 'positive ', 'starstarstarstar': 'positive ',
        '3star': 'negative ','3 sao': 'negative ','3sao': 'negative ', 'starstarstar': 'negative ',
        '2star': 'negative ','2 sao': 'negative ','2sao': 'negative ', 'starstar': 'negative ',
        '1star': 'negative ','1 sao': 'negative ','1sao': 'negative ', 'star': 'negative ',
        '0star': 'negative ','0 sao': 'negative ','0sao': 'negative ',
        '10 điểm' : 'positive ', ' 10đ' : 'positive ', 'mười điểm' : 'positive ', 'mừi điểm' : 'positive ',
        '9 điểm' : 'positive ', ' 9đ' : 'positive ', 'chín điểm' : 'positive ',
        '8 điểm' : 'positive ', ' 8đ' : 'positive ', 'tám điểm' : 'positive ',
        '7 điểm' : 'positive ', ' 7đ' : 'positive ', 'bảy điểm' : 'positive ',
        '6 điểm' : 'negative ', ' 6đ' : 'negative ', 'sáu điểm' : 'negative ',
        '5 điểm' : 'negative ', ' 5đ' : 'negative ', 'năm điểm' : 'negative ',
        '4 điểm' : 'negative ', ' 4đ' : 'negative ', 'bốn điểm' : 'negative ',
        '3 điểm' : 'negative ', ' 3đ' : 'negative ', 'ba điểm' : 'negative ',
        '2 điểm' : 'negative ', ' 2đ' : 'negative ', 'hai điểm' : 'negative ',
        '1 điểm' : 'negative ', ' 1đ' : 'negative ', 'một điểm' : 'negative ',
        '0 điểm' : 'negative ', ' 0đ' : 'negative ', 'không điểm' : 'negative ',
        }
    for star, label in stars_map.items():
        text = text.replace(star, f'SCORE{label.upper()}' )
        
    text = ' ' .join(text.split())
    return text


In [16]:
from num2words import num2words

def handle_number(text):
    # Xử lý chuyển số thành văn bản
    words = text.split()
    cleaned_words = []
    for word in words:
        try:
            # Nếu từ kết thúc bằng dấu chấm, loại bỏ dấu chấm và xử lý số
            if word.endswith( '.' ):
                num = int(word.replace( ',' , '' )[:-1])
                word = num2words(num, lang= 'vi' ) + '.' 
            else:
                # Nếu từ chứa dấu phẩy, loại bỏ dấu phẩy và xử lý số
                if ',' in word:
                    word = num2words(float(word), lang= 'vi' )
                elif '.' in word:
                    parts = word.split( '.' ) 
                    num = '' .join(parts[0:])
                    word = num2words(int(num), lang= 'vi' )
                else:
                    num = int(word)
                    word = num2words(num, lang= 'vi' )
        except ValueError:
            # Nếu không thể chuyển đổi, giữ nguyên từ
            pass
        cleaned_words.append(word)
    
    # Kết hợp các từ thành câu
    cleaned_text = ' ' .join(cleaned_words)
    
    return cleaned_text

In [17]:
#Lowercase
def lowercase(text):
    return text.lower()

## 3. Word segmentation

In [18]:
from pyvi import ViTokenizer
def Word_segmentation(text):
    text = ViTokenizer.tokenize(text)
    return text

## 4. Handle negation

In [19]:
def load_sentiment_lexicon(file_path):
    #Xử lý phủ định
    not_words = {"không", 'không_hề' , "chẳng", "chưa", "không_phải", "chả", "mất", 
                 "thiếu", "vô", "đếch", "đéo", "kém", "nỏ", "not",
                 "bớt", "không_bao_giờ",
                }
    # Tạo tập hợp chứa các từ tích cực
    positive_words = {
        "ưng_ý", "ưng", "kỹ", "được", "ô_kê", "ok", "mịn", "ổn", "xinh", "chúc_mừng",
        "hạnh_phúc", "sang", "oách", "khen", "ổn_định", "cảm_ơn", "cám_ơn", "chuẩn",
        "hoàn_thiện", "chắc_chắn", "sạch_sẽ", "hài_lòng", "chất_lượng", "hấp_dẫn",
        "vui_vẻ", "nguyên_chất", "thuận_lợi", "có_lợi", "tích_cực", "khuyến_khích",
        "tốt_hơn", "vị_tha", "sắc", "bén", "thích_hợp", "quý_báu", "sâu_sắc",
        "thịnh_vượng", "xinh_đẹp", "rực_rỡ", "trong_sáng", "chấp_nhận_được", "khéo_léo",
        "nghệ_thuật", "yên_tâm", "uyển_chuyển", "sôi_động", "nhân_đạo", "thân_mật",
        "thoải_mái", "đặc_biệt", "toàn_diện", "hòa_đồng", "hài_hòa", "thuận_tiện",
        "lịch_sự", "may_mắn", "may", "đoan_trang", "phấn_chấn", "sành_điệu", "sáng_suốt",
        "kín_đáo", "mát_mẻ", "lấp_lánh", "danh_dự", "dễ_dàng", "say_mê", "nhiệt_tình",
        "đạo_đức", "trung_thực", "trung_thành", "chung_thủy", "ngon", "chu_đáo", "ngăn_nắp",
        "lành_mạnh", "hợp_vệ_sinh", "khôn", "khen_ngợi", "quý_giá", "kháng_khuẩn", "êm_tai",
        "tinh_túy", "du_dương", "bổ_ích", "hồng_hào", "khỏe_khoắn", "khỏe_mạnh", "khỏe",
        "mạnh", "săn_chắc", "sung_sức", "mạnh_khỏe", "trẻ_trung", "đùa", "đề_cao", "quản_lý",
        "cánh_tay_phải", "nhận_dạng_được", "hoàn_hảo", "trọn_vẹn", "lý_tưởng", "dễ_an_ủi",
        "đẹp", "duyên_dáng", "tuyệt_vời", "đáng_ngưỡng_mộ", "thú_vị", "ngọt_ngào", "lạc_quan",
        "sinh_lợi", "chính_đáng", "khiêm_tốn", "minh_mẫn", "uy_tín", "vinh_dự", "thẳng_thắn",
        "bảo_đảm", "màu_mỡ", "dễ_chịu", "tươi", "cẩn_thận", "đúng", "hiệu_quả", "cute",
        "dễ_thương", "phê", "xịn", "sịn", "phê", "vui_tính", "chính_hãng", "thực_sự",
        "vinh_quang", "thánh_thiện", "vui_tươi", "gợi_cảm", "cân_đối", "chân_thành",
        "thành_thạo", "tinh_tế", "kiên_cố", "thân_thiện", "thích", "tỏa_sáng", "ngưỡng_mộ",
        "phù_hợp", "hy_vọng", "tốt_đẹp", "tốt", "đẹp", "giỏi_giang", "lôi_cuốn", "uyên_bác",
        "yêu", "thích_thú", "ái_ân", "chân_tình", "chăm_chút", "tuyệt", "nhẹ_nhõm", "xinh_xắn",
        "giỏi", "khủng", "đạt", "khỏe", "hợp_lý", "hợp_lí", "sạch", "ấm", "mềm", "cải_thiện",
        "tiện", "gọn", "uy_tín", "tin_tưởng", "đẹp", "hạnh_phúc", "nhạy", "nhạy_bén",
        "pin_rất_trâu", "bao_mượt", "pin_trâu", "hài_lòng", "sạc_nhanh", "đẹp", "tuyệt_vời"
    }

    negative_words = {
        "bất_lợi", "chán", "chật_hẹp", "chật", "tức_giận", "xấu", "khủng_khiếp", "mỏng", "nhầm", "đe_dọa",
        "ghê", "hiểm_ác", "lừa_dối", "lừa", "mặn", "tệ_nhất", "bẩn_thỉu", "hà_khắc", "cay", "ngu_dốt", "hiếm",
        "ngược_đãi", "chậm", "căng_thẳng", "thô_bạo", "khó_chịu", "khắc_nghiệt", "kị", "ghen", "hỗn_tạp", "dơ",
        "liều_lĩnh", "dơ_bẩn", "thô_tục", "tệ_hại", "tệ", "nhầm_lẫn", "quá_mức", "xấu_số", "ngu_si", "đau_đớn",
        "phàn_nàn", "phản_cảm", "tàn_phá", "bất_mãn", "hung_hăng", "bất_tiện", "hoang_sơ", "bẩn_thỉu", "dơ_bẩn",
        "giả_dối", "đắt_đỏ", "đắt", "yếu", "sai_lầm", "lầm", "nghiêm_trọng", "đáng_ghét", "hỏng", "bất_hợp_tác",
        "chán_nản", "yếu_đuối", "trục_trặc", "bực_bội", "tàn_bạo", "bừa_bãi", "lăng_nhăng", "thất_vọng", "chê_bai",
        "loang_lổ", "tiêu_hao", "bất_công", "lang_thang", "khổ_sở", "vớ_vẩn", "bất_hạnh", "vô_tâm", "bù_xù",
        "bừa_bộn", "khó", "gian_dối", "vô_dụng", "vô_nghĩa", "ác", "chóng_mặt", "là_lạ", "miễn_cưỡng", "ngu_ngốc",
        "dị_ứng", "co_cứng", "hại", "lạm_dụng", "vu_khống", "tai_hại", "tồi", "xảo_quyệt", "đau_thương", "hỗn_loạn",
        "nhức_nhối", "đỏ_ngầu", "loét", "sưng_tấy", "tấy", "viêm", "ốm_yếu", "khô", "nặng_bụng", "nặng_nề", "khàn_khàn",
        "dị", "lật", "vô_vọng", "gian_lận", "xuống_cấp", "ứ_đọng", "lạnh_toát", "oi_ả", "sưng", "bị_nhọt", "có_ác_cảm",
        "tàn_nhẫn", "mù_quáng", "bất_thường", "bất_tín", "gay_gắt", "mất_lòng", "bạc_bẽo", "thô", "thất_sách",
        "quái_đản", "thù_địch", "xúc_phạm", "bất_trị", "yếu_đuối", "run", "gây_mê", "cạn_kiệt", "tàn_tật", "định_mệnh",
        "hôi_thối", "mốc", "hôi", "gẫy", "lởm", "hắc", "dỏm", "giởm", "dởm", "nhòe", "chết", "móp", "mùi_thối", "thối",
        "ràng_buộc", "hư_hỏng", "bị", "hư", "giả_mạo", "giả_tạo", "giả", "sợ_hãi", "khó_khăn", "xấu", "bốc_mùi",
        "hôi_thối", "dã_man", "nham_hiểm", "tham_nhũng", "xấu_xa", "ủ_rũ", "thâm", "kích_ứng", "hờn_dỗi", "vu_khống",
        "bôi_nhọ", "tác_hại", "tinh_nghịch", "khó_tiêu", "thong_thả", "nhàn_nhã", "trơ", "thối_rữa", "phù_phiếm",
        "độc_quyền", "do_dự", "nạn_nhân", "rắc_rối", "sai", "định_kiến", "buồn_bã", "bứt_rứt", "mùi", "bại_hoại",
        "giận_dữ", "báo_động", "phẫn_nộ", "ghét", "kênh_kiệu", "nhàm_chán", "buồn", "xót_xa", "đau_lòng", "khủng_khiếp",
        "1star", "2star", "ngắn", "ác", "tổn_thất", "nặng_nề", "xót_xa", "đau_lòng", "bức_xúc", "tàn_ác", "ghét",
        "ác_hiểm", "rởm", "tróc", "ám_sát", "lười", "vụn", "gãy", "hối_tiếc", "tiêu_cực", "ngu", "đắt", "hốt_hoảng",
        "đểu", "nhái", "ngứa", "cùi", "hàng_lô", "hàng_giả", "phức_tạp", "nát", "mờ", "đơ", "ngỏm", "thất_vọng",
        "lâu", "nặng", "chậm", "thủng", "hỏng", "trầy", "dão", "lỗi", "kém", "lùn", "buồn", "bùn", "thiếu",
        "thất_vọng", "sai", "rách", "ngấy", "tồi_tệ", "mẻ", "ẩu", "cẩu_thả", "lộn", "phức_tạp", "ế_ẩm", "ế",
        "sướt", "kém", "tốn_pin", "nóng_máy", "nóng", "thất_vọng", "giật_lag", 
    }

    sentiment_lexicon = {}
    
    raw_url = file_path
    response = requests.get(raw_url)

    if response.status_code == 200:
        content = response.text
    else:
        print('Failed to retrieve the file from GitHub:', response.status_code)

    header_skipped = False
    for line in content.split('\n'):
        if not header_skipped:
            if "POS\tID\tPosScore\tNegScore\tSynsetTerms\tGloss" in line:
                header_skipped = True
            continue
        parts = line.strip().split("\t")
        if len(parts) >= 5:
            word = parts[4]
            pos_score = float(parts[2])
            neg_score = float(parts[3])
            if pos_score > 0.5:
                sentiment_lexicon[word.split( '#' )[0]] = "positive"
            if neg_score > 0.5:
                sentiment_lexicon[word.split( '#' )[0]] = "negative"

    # Bổ sung từ điển với các từ tích cực và tiêu cực
    for word in positive_words:
        sentiment_lexicon[word] = "positive"
    for word in negative_words:
        sentiment_lexicon[word] = "negative"

    return not_words, positive_words, negative_words, sentiment_lexicon




from pyvi import ViTokenizer

def handle_negation(text, sentiment_lexicons):
    not_words, positive_words, negative_words, sentiment_lexicon = sentiment_lexicons
    texts = text.split()
    len_text = len(texts)

    i = 0
    while i < len_text:
        cp_text = texts[i]
        if i < len_text - 1:
            combine_word = cp_text + '_' + texts[i+1]
            if combine_word in not_words:
                numb_word = min(3, len_text - i - 2)
                for j in range(numb_word):
                    combine_word_next = texts[i + j + 2]
                    if combine_word_next in positive_words:
                        texts[i] = 'NOTPOS'
                        texts[i + j + 1] = ''
                        texts[i + j + 2] = ''
                    elif combine_word_next in negative_words:
                        texts[i] = 'NOTNEG'
                        texts[i + j + 1] = ''
                        texts[i + j + 2] = ''
                    else:
                        break
                i += numb_word 
            elif cp_text in not_words: 
                numb_word = min(2, len_text - i - 1)
                for j in range(numb_word):
                    if texts[i + j + 1] in positive_words:
                        texts[i] = 'NOTPOS'
                        texts[i + j + 1] = ''
                    elif texts[i + j + 1] in negative_words:
                        texts[i] = 'NOTNEG'
                        texts[i + j + 1] = ''
                    else:
                        break
                i += numb_word
            i+=1
        else: 
            if cp_text in positive_words:
                texts.append('POSITIVE')
            elif cp_text in negative_words:
                texts.append('NEGATIVE')
            i += 1

    text = ' '.join(texts)
    return text
    

In [20]:
from num2words import num2words

def process_text(text):
    cleaned_text = clean_text(text)
    cleaned_text = handle_number(cleaned_text)

    cleaned_text = remove_punctuation(cleaned_text)

    cleaned_text = lowercase(cleaned_text)
    cleaned_text = process_emojis(cleaned_text)
    
    
    cleaned_text = process_emojis(cleaned_text)
    cleaned_text = cleaning_repeating_char(cleaned_text)
    
    cleaned_text = remove_elongated_chars(cleaned_text)
    cleaned_text = normalize_unicode(cleaned_text)
    cleaned_text = normalize_stars(cleaned_text)
    cleaned_text = normalize_sentiment_words(cleaned_text)
    
    cleaned_text = Word_segmentation(cleaned_text)

    file_path = "https://raw.githubusercontent.com/tiennm0510/server_python/master/VietSentiWordnet_ver1.0.txt"
    sentiment_lexicon = load_sentiment_lexicon(file_path)
    cleaned_text = handle_negation(cleaned_text, sentiment_lexicon)

    #remove nốt những ký tự thừa thãi
    cleaned_text = cleaned_text.replace(u'  ', u' ')
    cleaned_text = cleaned_text.replace(u'"', u' ')
    cleaned_text = cleaned_text.replace(u'️', u'')
    
    cleaned_text = remove_stopword(cleaned_text)
    
    return cleaned_text

In [21]:
text = 'Mọi người 1 cập nhật phần mềm lại , nó sẽ bớt tốn pin, mình đã thử rồi, mọi thứ cũng ok, nhưng vân tay ko nhạy'
process_text(text)

'cập_nhật phần_mềm bớt tốn pin thử ok vân NOTPOS'

## 5. Xử lí label

In [22]:
import pandas as pd

def process_label(label):
    label = label.replace("{", "")
    parts = label.strip("}").split(";")
    categories = []
    sentiments = []
    for part in parts:
        if "#" in part:
            category, sentiment = part.split("#")
            sentiment = sentiment.replace("}", "")
            categories.append(category)
            sentiments.append(sentiment)
    return pd.Series([categories, sentiments])

In [25]:
df = pd.read_csv('/kaggle/input/uit-visfd/Train.csv')
df["comment"] = df["comment"].apply(process_text)
df[["category", "sentiment"]] = df["label"].apply(process_label)
df['sentiment'] = df['sentiment'].apply(lambda x: ', '.join(x))
df['category'] = df['category'].apply(lambda x: ', '.join(x))
# executed in 23 mins

In [27]:
data_without_diacritics = df.copy()
data_without_diacritics["comment"] = data_without_diacritics["comment"].apply(remove_diacritics)
augmented_data = pd.concat([df, data_without_diacritics], ignore_index=True)

In [28]:
augmented_data.to_csv('Processed_train.csv')

In [30]:
df = pd.read_csv('/kaggle/input/uit-visfd/Test.csv')
df["comment"] = df["comment"].apply(process_text)
df[["category", "sentiment"]] = df["label"].apply(process_label)
df['sentiment'] = df['sentiment'].apply(lambda x: ', '.join(x))
df['category'] = df['category'].apply(lambda x: ', '.join(x))
# df.to_csv('Processed_test.csv')

In [31]:
data_without_diacritics = df.copy()
data_without_diacritics["comment"] = data_without_diacritics["comment"].apply(remove_diacritics)
augmented_data = pd.concat([df, data_without_diacritics], ignore_index=True)

In [32]:
augmented_data.to_csv('Processed_test.csv')

In [33]:
df = pd.read_csv('/kaggle/input/uit-visfd/Dev.csv')
df["comment"] = df["comment"].apply(process_text)
df[["category", "sentiment"]] = df["label"].apply(process_label)
df['sentiment'] = df['sentiment'].apply(lambda x: ', '.join(x))
df['category'] = df['category'].apply(lambda x: ', '.join(x))
# df.to_csv('Processed_dev.csv')

In [34]:
data_without_diacritics = df.copy()
data_without_diacritics["comment"] = data_without_diacritics["comment"].apply(remove_diacritics)
augmented_data = pd.concat([df, data_without_diacritics], ignore_index=True)

In [35]:
augmented_data.to_csv('Processed_dev.csv')

In [ ]:
def process_data_for_absa(data):
    """
    Xử lý dữ liệu cho ABSA.

    Args:
        data (pd.DataFrame): DataFrame chứa dữ liệu đã được gắn nhãn aspect và sentiment.

    Returns:
        pd.DataFrame: DataFrame đã xử lý cho ABSA.
    """
    processed_data = []
    for index, row in data.iterrows():
        comment = row["comment"]
        aspects = row["category"].split(", ")  # Tách các aspect
        sentiments = row["sentiment"].split(", ")  # Tách các sentiment
        # Tạo một sample cho mỗi cặp aspect-sentiment
        for aspect, sentiment in zip(aspects, sentiments):
            processed_data.append({
                "comment": comment,
                "aspect": aspect.strip(),
                "sentiment": sentiment.strip()
            })
    return pd.DataFrame(processed_data)


In [ ]:
# processed_data = process_data_for_absa(df)
# processed_data